In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("test").getOrCreate()
from pyspark.sql.functions import format_string
import sys

In [ ]:
#Create a single dataframe from all CSV files in the zip, with header information
df1 = spark.read.format("csv").options(header = 'true', inferSchema = 'true').load("C:/Users/jack_/Desktop/Big_data/drive_stats_2019_Q1/*.csv")
df1.createOrReplaceTempView("table1")

In [16]:
#Show the dataframe columns
df = df1.columns
print(df)

['date', 'serial_number', 'model', 'capacity_bytes', 'failure', 'smart_1_normalized', 'smart_1_raw', 'smart_2_normalized', 'smart_2_raw', 'smart_3_normalized', 'smart_3_raw', 'smart_4_normalized', 'smart_4_raw', 'smart_5_normalized', 'smart_5_raw', 'smart_7_normalized', 'smart_7_raw', 'smart_8_normalized', 'smart_8_raw', 'smart_9_normalized', 'smart_9_raw', 'smart_10_normalized', 'smart_10_raw', 'smart_11_normalized', 'smart_11_raw', 'smart_12_normalized', 'smart_12_raw', 'smart_13_normalized', 'smart_13_raw', 'smart_15_normalized', 'smart_15_raw', 'smart_16_normalized', 'smart_16_raw', 'smart_17_normalized', 'smart_17_raw', 'smart_22_normalized', 'smart_22_raw', 'smart_23_normalized', 'smart_23_raw', 'smart_24_normalized', 'smart_24_raw', 'smart_168_normalized', 'smart_168_raw', 'smart_170_normalized', 'smart_170_raw', 'smart_173_normalized', 'smart_173_raw', 'smart_174_normalized', 'smart_174_raw', 'smart_177_normalized', 'smart_177_raw', 'smart_179_normalized', 'smart_179_raw', 'sma

In [17]:
#Show the first 20 rows, sorted by (capacity descending, model ascending)
top20 = spark.sql("select * from table1 order by capacity_bytes desc, model asc limit 20")
top20.show()

+----------+-------------+-------------------+--------------+-------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+-------------------+------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+---

In [3]:
#Count the total number of rows
count = spark.sql("select count(*) as Total_Rows from table1")
count.show()

+----------+
|Total_Rows|
+----------+
|   9577046|
+----------+



In [14]:
#Count the total number of rows, grouped by capacity
count1 = spark.sql("select capacity_bytes, count(*) as Total_Rows from table1 group by capacity_bytes")
count1.show()

+--------------+----------+
|capacity_bytes|Total_Rows|
+--------------+----------+
|14000519643136|    109378|
| 2000398934016|       265|
|  500107862016|    174254|
|  250059350016|       112|
|  480036847616|       540|
| 4000787030016|   3499483|
| 1000204886016|        90|
| 8001563222016|   2280970|
|  320072933376|       117|
| 5000981078016|      4050|
|  137438952960|         1|
|10000831348736|    109800|
|  160041885696|       307|
|12000138625024|   3228204|
|            -1|       993|
| 6001175126016|    168482|
+--------------+----------+



In [4]:
#Get the dataframe summary statistics
data = spark.sql("select * from table1 limit 1000000")
dfSummary = data.summary()
dfSummary.show()

+-------+----------+----------------+------------+--------------------+--------------------+------------------+-------------------+------------------+-----------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+--------------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+-----------------+--------------------+------------------+-------------------+------------+-------------------+------------+-------------------+------------------+-------------------+------------------+-------------------+------------------+-------------------+------------+-------------------+------------+--------------------+-------------+--------------------+------------------+--------------------+-------------------+--------------------+------------------+--------------------+------------------+--------------------+-------------+----

In [5]:
#Select the following columns: date, model, capacity
sel = spark.sql("select date, model, capacity_bytes from table1")
sel.show()

+----------+--------------------+--------------+
|      date|               model|capacity_bytes|
+----------+--------------------+--------------+
|2019-03-05|         ST4000DM000| 4000787030016|
|2019-03-05|       ST12000NM0007|12000138625024|
|2019-03-05|       ST12000NM0007|12000138625024|
|2019-03-05|       ST12000NM0007|12000138625024|
|2019-03-05|HGST HMS5C4040ALE640| 4000787030016|
|2019-03-05|        ST8000NM0055| 8001563222016|
|2019-03-05|       ST12000NM0007|12000138625024|
|2019-03-05|       ST12000NM0007|12000138625024|
|2019-03-05|       ST12000NM0007|12000138625024|
|2019-03-05|        ST8000NM0055| 8001563222016|
|2019-03-05|         ST4000DM000| 4000787030016|
|2019-03-05|         ST8000DM002| 8001563222016|
|2019-03-05|       ST12000NM0007|12000138625024|
|2019-03-05|        ST8000NM0055| 8001563222016|
|2019-03-05|       ST12000NM0007|12000138625024|
|2019-03-05|HGST HMS5C4040ALE640| 4000787030016|
|2019-03-05|HGST HMS5C4040BLE640| 4000787030016|
|2019-03-05|HGST HMS

In [9]:
#Select the number of distinct models
models = spark.sql("select distinct model from table1")
models.createOrReplaceTempView("table2")
num = spark.sql("select count(*) as Total_Distinct_Models from table2")
num.show()

+---------------------+
|Total_Distinct_Models|
+---------------------+
|                   49|
+---------------------+



In [10]:
#Calculate the pairwise frequency of this two columns (e.g. crosstab): capacity, smart_1_normalized
table = spark.sql("select * from table1")
pairwise = table.crosstab('capacity_bytes', 'smart_1_normalized')
pairwise.show()

+---------------------------------+-------+----+-----+----+----+-----+-----+-----+-----+-----+-----+------+-----+-----+------+------+------+------+------+------+-----+---+---+---+---+---+-----+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+------+------+------+------+------+------+------+------+------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+----+
|capacity_bytes_smart_1_normalized|    100| 101|  102| 103| 104|  105|  106|  107|  108|  109|  110|   111|  112|  113|   114|   115|   116|   117|   118|   119|  120|149|153|160|187|199|  200| 49| 50| 51| 53| 54| 55| 56| 57| 58| 59| 60| 61| 62| 63|  64|  65|   66|   67|   68|   69|   70|   71|   72|   73|   74|   75|    76|    77|    78|    79|    80|    81|    82|    83|    84| 85| 86| 87| 88| 89| 90| 91| 92| 93| 94| 95| 96| 97| 98|  99|null|
+---------------------------------+-------+----+-----+----+----+-----+-----+-----+-----+-----+-----+--

In [15]:
#Find the mean value of column capacity
mean = spark.sql("select avg(capacity_bytes) as mean from table1")
mean.show()

+--------------------+
|                mean|
+--------------------+
|7.804058925479401E12|
+--------------------+

